# Big Query Operations

In [1]:
#!pip install --upgrade 'google-cloud-bigquery[pandas]'

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import Conflict

In [2]:
#path to your service account credentials

SERVICE_ACCOUNT_PATH='cred/big-query-service-acct.json' #update service account path

In [3]:
# create gcp client by providing credentials and project_id to interact with BigQuery

credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_PATH)
project_id = '**********' #update project_id as per your requirement
client = bigquery.Client(credentials= credentials,project=project_id)

In [4]:
#Query data from BigQuery Tables

QUERY ='''
        SELECT * FROM `bigquery-public-data.usa_names.usa_1910_2013`
        WHERE state = "TX"
        LIMIT 10
'''
query_job = client.query(QUERY)

for row in query_job.result():
    print(row)

Row(('TX', 'F', 1910, 'Frances', 197), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Alice', 149), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Beatrice', 123), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Ella', 102), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Gertrude', 97), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Josephine', 86), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Lula', 77), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Blanche', 50), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Marjorie', 40), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4})
Row(('TX', 'F', 1910, 'Christine', 34), {'state': 0, 'gender': 1, 'year': 2, 'name': 3, 'number': 4

In [5]:
#Convert BigQuery result to Pandas DataFrame

dataframe = query_job.result().to_dataframe()
dataframe

,state,gender,year,name,number
0,TX,F,1910,Frances,197
1,TX,F,1910,Alice,149
2,TX,F,1910,Beatrice,123
3,TX,F,1910,Ella,102
4,TX,F,1910,Gertrude,97
5,TX,F,1910,Josephine,86
6,TX,F,1910,Lula,77
7,TX,F,1910,Blanche,50
8,TX,F,1910,Marjorie,40
9,TX,F,1910,Christine,34


In [6]:
def create_data_set(bq_client,dataset_name,dataset_location='US'):
    '''
    Create Dataset in BigQuery 
    
    bq_client : BigQuery Client to interact with big query services
    dataset_name : name of dataset
    dataset_location : location of your dataset; defaults to US
    '''
    
#     dataset = bigquery.Dataset(dataset_name)

    try:
        dataset = bq_client.create_dataset(dataset_name, timeout=30)
        print("Created dataset {}.{}".format(dataset.project, dataset.dataset_id))
        
    except Conflict:
        print("Dataset already exist")

    except Exception as err:
        print(type(err).__name__)
        print("Exception occured")
        print(err)

In [7]:
def createTable(bq_client,dataset_name,table_name,schema):
    '''
    Create table in BigQuery 
    
    bq_client : BigQuery Client to interact with big query services
    dataset_name : name of dataset
    table_name : name of table
    schema : schema of the table
    '''
    
    table_id = f"{bq_client.project}.{dataset_name}.{table_name}"
    table = bigquery.Table(table_id, schema=schema)
    
    try:
        table = bq_client.create_table(table)
        print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")

    except Conflict:
        print("Table already exist")

    except Exception as err:
        print(type(err).__name__)
        print("Exception occured")

In [8]:
#creating dataset in bigquery

DATASET_NAME='python_dataset2' #update dataset_name as per your requirement

create_data_set(client,DATASET_NAME) #create dataset

client.get_dataset(DATASET_NAME) #check if dataset is created in project

Created dataset triple-access-349104.python_dataset2


Dataset(DatasetReference('triple-access-349104', 'python_dataset2'))

In [9]:
#creating table in bigquery

TABLE_NAME='demo_table2' #update table_name as per your requirement

schema = [
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("age", "INTEGER"),
    bigquery.SchemaField("city", "STRING"),
]

createTable(client,DATASET_NAME,TABLE_NAME,schema) #create table

client.get_table(f"{DATASET_NAME}.{TABLE_NAME}") #check if table is created in Dataset

Created table triple-access-349104.python_dataset2.demo_table2


Table(TableReference(DatasetReference('triple-access-349104', 'python_dataset2'), 'demo_table2'))

In [10]:
#checking total size processed by a query

job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)

query_job = client.query('''
    SELECT name, COUNT(*) as name_count
    FROM `bigquery-public-data.usa_names.usa_1910_2013`
    WHERE state = 'WA'
    GROUP BY name''',
    job_config=job_config,
)

print(f"This query will process {query_job.total_bytes_processed/2**20:.2f} MB.")

This query will process 62.88 MB.
